Copyright 2024 - Forusone : shins777@gmail.com

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# Search context from structured data store on Vertex AI Search

# Configuration
## Install python packages


In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform \
                               google-cloud-discoveryengine

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 30.9 MB/s eta 0:00:00


## Authentication to access to the GCP
* Use OAuth to access the GCP environment.
 * Refer to the authentication methods in GCP : https://cloud.google.com/docs/authentication?hl=ko

In [2]:
import sys
from IPython.display import Markdown, display

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id="ai-hangsik")

!gcloud config set project ai-hangsik

Updated property [core/project].


## Set the environment on GCP Project


In [3]:
MODEL_NAME="gemini-1.5-flash"
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"

### Vertex AI initialization
Configure Vertex AI and access to the foundation model.
* Vertex AI initialization : aiplatform.init(..)
  * https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization

In [4]:
import vertexai

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    Tool,
)

from vertexai.preview.generative_models import grounding

vertexai.init(project=PROJECT_ID, location=REGION)
model = GenerativeModel(MODEL_NAME)

In [5]:
SEARCH_APP_LOCATION = "global"
SEARCH_ENGINE_ID = "bbc-news_1732530458578"

### Helper functions to search and parse context

In [10]:

def search_context(query:str):
  """
  Function to search contexts with search configuration.

  Args:
    query : query to be sent to the search engine.

  Returns:
    The response from the search engine.

  """
  from google.api_core.client_options import ClientOptions
  # from google.cloud import discoveryengine_v1alpha as discoveryengine
  from google.cloud import discoveryengine

  # Create a client using a regional endpoint
  client = discoveryengine.SearchServiceClient(
      client_options=(
          ClientOptions(
              api_endpoint=f"{SEARCH_APP_LOCATION}-discoveryengine.googleapis.com"
          )
          if SEARCH_APP_LOCATION != "global"
          else None
      )
  )

  # The full resource name of the search app serving config
  serving_config = f"projects/{PROJECT_ID}/locations/{SEARCH_APP_LOCATION}/collections/default_collection/engines/{SEARCH_ENGINE_ID}/servingConfigs/default_config"

  response = client.search(
      # [SearchRequest] https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.types.SearchRequest
      discoveryengine.SearchRequest(
          serving_config=serving_config,
          query=query,
          page_size=2,
      )
  )

  return response

#-------------------------------------------------------

def parse_results(response):
  """Parse response to build a conext to be sent to LLM
     Structured datastore only.
  """

  import re
  retrieved_data: list[str] = []

  for result in response.results:
    for key, value in result.document.struct_data.items():
      #print(f"Field: {key}, Value: {value}")
      if key =="body":
        retrieved_data.append(value)
        print(value)

  return  retrieved_data

#-------------------------------------------------------

def generate(query:str):
    """
    Generate a response from the model.

    query :
      query to be sent to the model

    Returns:
      The generated response.

    """

    # Set model parameter : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-multimodal-prompts#set_model_parameters
    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 1,
        "top_p": 0.95,
    }

    responses = model.generate_content(
        [query],
        generation_config=generation_config,
        stream=False,
    )

    return responses.text

### Run example

In [11]:
from time import perf_counter

t1_start = perf_counter()

query = "what is the Kyrgyz Republic?"
response = search_context(query)
retrieved_data = parse_results(response)

prompt = f"""

  당신은 AI 어시스턴트입니다.
  아래 Question 에 대해서 반드시 Context에 있는 개별 내용을 기반으로 단계적으로 추론해서 근거를 설명하고 답변해주세요.
  Context : {retrieved_data}
  Question : {query}
  """

outcome = generate(prompt)

t1_end  = perf_counter()
print(f"Time : {t1_end - t1_start} seconds\n\n")

display(Markdown(outcome))


The Kyrgyz Republic, a small, mountainous state of the former Soviet republic, is using invisible ink and ultraviolet readers in the country's elections as part of a drive to prevent multiple voting.

This new technology is causing both worries and guarded optimism among different sectors of the population. In an effort to live up to its reputation in the 1990s as "an island of democracy", the Kyrgyz President, Askar Akaev, pushed through the law requiring the use of ink during the upcoming Parliamentary and Presidential elections. The US government agreed to fund all expenses associated with this decision.

The Kyrgyz Republic is seen by many experts as backsliding from the high point it reached in the mid-1990s with a hastily pushed through referendum in 2003, reducing the legislative branch to one chamber with 75 deputies. The use of ink is only one part of a general effort to show commitment towards more open elections - the German Embassy, the Soros Foundation and the Kyrgyz gover

The Kyrgyz Republic is a small, mountainous state of the former Soviet republic. 
